# 1. Build Team Identities, Company Employment and User Identities

Notebook to build all of the different identities assumed available when I have initiailised Type Refinery. Note that the Stix objects created by these classes are frozen dicts, so its better to create the components and then the whole, rather than create an empty whole first and then add the components, which requires updating.

We will take the Type Refinery User as an Example of the Process of making a group of Complex Identity using the official [Identity_Contact_Information Extension](https://github.com/os-threat/cti-stix-common-objects/tree/main/extension-definition-specifications/identity-contact-information) specification, and combining it with the standard Stix approach. There are three stages:

1. Create the `User_Account` first,
2. Then the `Email_Address` objects, attaching the new User_Account.["id"]
3. Then create the `EmailContact`, `SocialMediaContact`, `ContactNumber` lists of sub-objects from the specification, attaching their User_Account.["id"] and Email_Address.["id"] as required
4. Create the [Identity Stix object](https://docs.oasis-open.org/cti/stix/v2.1/os/stix-v2.1-os.html#_wh296fiwpklp) with the [Identity_Contact_Information Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/identity-contact-information/Identity%20Contact%20Information.docx), and joining the sub objects to the Extension object and then down to the base Identity object
5. Collect all of the objects and put them into a bundle, and save into a json

Then we will create identities fro the three dummy users that are always clicking the wrong links, but at least they are good on reporting. Finally, we will employ the four Identities with our current company, and save the complete group of objects into a bundle

Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (It also verifies the objects are created correctly, and is cool).

## A. Load Imports

### A.1 StixORM Imports


In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle
)
from stixorm.module.definitions.os_threat import (
    IdentityContact, EmailContact, SocialMediaContact, ContactNumber
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

## A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [2]:
import sys
sys.path.append('../')
import os

## A.3 Relative Import of Blocks

In [3]:
import json
from Block_Families.Objects.SCO.User_Account.make_user_account import main as make_user_account
from Block_Families.Objects.SCO.Email_Addr.make_email_addr import main as make_email_addr
from Block_Families.Objects.SDO.Identity.make_identity import main as make_identity
from Block_Families.Objects.SRO.Relationship.make_sro import main as make_sro
from utilities import emulate_ports, unwind_ports

path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_foundation.json"
identities_data = [
    {
        "who": "me", "email": "SCO/Email_Addr/email_addr_TR_user.json", 
        "acct": "SCO/User_Account/usr_account_TR_user.json",
        "ident": "SDO/Identity/identity_TR_user.json",
        "results": "TR_Identity"
    }, 
    {
        "who": "usr1", "email": "SCO/Email_Addr/email_addr_IT_user1.json", 
        "acct": "SCO/User_Account/usr_account_IT_user1.json",
        "ident": "SDO/Identity/identity_IT_user1.json",
        "results": "Usr1"
    }, 
    {
        "who": "usr2", "email": "SCO/Email_Addr/email_addr_IT_user2.json", 
        "acct": "SCO/User_Account/usr_account_IT_user2.json",
        "ident": "SDO/Identity/identity_IT_user2.json",
        "results": "Usr2"
    }, 
    {
        "who": "usr3", "email": "SCO/Email_Addr/email_addr_IT_user3.json", 
        "acct": "SCO/User_Account/usr_account_IT_user3.json",
        "ident": "SDO/Identity/identity_IT_user3.json",
        "results": "Usr3"
    }, 
]
company_base = {
    "ident": "SDO/Identity/identity_TR_user_company.json",
    "results": "employer"
}
# Make the Variable to hold the Bundle List
bundle_list = []
results_paths = {}

## B. Create the Multi-Faceted User of this Type Refinery

only perform once on initialisation, otherwise you must update the identity object

### B.1 First Select the User Data

In [4]:
for ident in identities_data:
    if ident["who"] == "me":
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]


### B.2 Make the User Account
Simply pass the paths for the input file and where the results should be written

In [5]:
def local_make_acct(acct_path, results_path):
    # Set the Relative Input and Output Paths for the block
    acct_data_rel_path =  path_base + acct_path
    acct_results_rel_path = results_base + results_path + "__acct.json"
    results_paths["acct"] = acct_results_rel_path
    # Run the Make User Account block
    make_user_account(acct_data_rel_path,acct_results_rel_path)
    # Retrieve the saved file
    if os.path.exists(acct_results_rel_path):
        with open(acct_results_rel_path, "r") as script_input:
            export_data = json.load(script_input)
            export_data_list = export_data["user-account"]
            stix_object = export_data_list[0]
            # convert it into a Stix Object and append to the bundle
            usr_acct = UserAccount(**stix_object)
            print(usr_acct.serialize(pretty=True))
            bundle_list.append(usr_acct)

# make the account object
local_make_acct(acct_path, results_path)

{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3",
    "user_id": "79563902",
    "account_login": "tjones",
    "account_type": "soc,",
    "display_name": "Trusty Jones"
}


### B.3 Make the Email Address

In [6]:
def local_make_email(email_path, results_path, acct_results):
    #
    # 1. Set the Relative Input and Output Paths for the block
    #
    email_data_rel_path = path_base + email_path
    email_results_rel_path = results_base + results_path + "__email.json"
    results_paths['email'] = email_results_rel_path
    #
    # NOTE: This code is only To fake input ports
    # Add the User Account object and the  EmailAddress
    #  Form data file
    #
    ports = []
    if acct_results:
        ports.append(acct_results)
    emulate_ports(email_data_rel_path, ports)
    #
    # Make the Email Address object
    #
    make_email_addr(email_data_rel_path,email_results_rel_path)
    #
    # Remove Port Emulation - Fix the data file so it only has form data
    #
    if ports:
        unwind_ports(email_data_rel_path)

    # Retrieve the saved file
    if os.path.exists(email_results_rel_path):
        with open(email_results_rel_path, "r") as script_input:
            export_data = json.load(script_input)
            export_data_list = export_data["email-addr"]
            stix_object = export_data_list[0]
            # convert it into a Stix Object and append to the bundle
            email_addr = EmailAddress(**stix_object)
            print(email_addr.serialize(pretty=True))
            bundle_list.append(email_addr)



# make the email object
email_results_rel_path=""
local_make_email(email_path, results_path, results_paths['acct'])

{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3",
    "value": "trusty@example.com",
    "display_name": "Trusty Jones",
    "belongs_to_ref": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3"
}


### B.4 Make the Identity

In [7]:
def local_make_identity(ident_path, results_path, email_results=None, acct_results=None):
    # Set the Relative Input and Output Paths for the block
    identity_data_rel_path = path_base + ident_path
    identity_results_rel_path = results_base + results_path + "__ident.json"
    #
    # NOTE: This code is only To fake input ports
    # Add the USer Account object and the  EmailAddress
    #  to the Identity Form object
    ##
    # NOTE: This code is only To fake input ports
    # Add the User Account object and the  EmailAddress
    #  Form data file
    #
    ports = []
    if email_results:
        ports.append(email_results)
    if acct_results:
        ports.append(acct_results)
    emulate_ports(identity_data_rel_path, ports)
    # # Make the Identity object
    # print(f"ports are -> {ports}")
    # print(f"identity_data_rel_path -> {identity_data_rel_path}")
    make_identity(identity_data_rel_path,identity_results_rel_path)
    #
    # Remove Port Emulation if used - Fix the data file so it only has form data
    #
    if ports:
        unwind_ports(identity_data_rel_path)
    # Retrieve the saved file
    if os.path.exists(identity_results_rel_path):
        with open(identity_results_rel_path, "r") as script_input:
            export_data = json.load(script_input)
            export_data_list = export_data["identity"]
            stix_object = export_data_list[0]
            # convert it into a Stix Object and append to the bundle
            identity = Identity(**stix_object)
            print(identity.serialize(pretty=True))
            bundle_list.append(identity)



# make the identity object
local_make_identity(ident_path, results_path, email_results=results_paths['email'], acct_results=results_paths['acct'])

{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--5ca6788a-f46a-4647-b563-9b951d4dd1a0",
    "created": "2023-12-13T14:20:17.443978Z",
    "modified": "2023-12-13T14:20:17.443978Z",
    "name": "Trusty Jones",
    "description": "A Trusty Individual",
    "roles": [
        "soc",
        "support"
    ],
    "identity_class": "individual",
    "sectors": [
        "technology"
    ],
    "extensions": {
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "contact_numbers": [
                {
                    "contact_number_type": "work-phone",
                    "contact_number": "0418-208-368"
                }
            ],
            "email_addresses": [
                {
                    "digital_contact_type": "work",
                    "email_address_ref": "email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3"
                }
            ],
            "first_nam

## C. Employ the User

only perform once on initialisation, otherwise you must update the identity object

### C.1 First Create the Company

In [8]:
cwd = os.getcwd()
#
# Setup company info into paths
company_ident_path = company_base["ident"]
company_results_path = company_base["results"]
#
# MAke the Company object nd add it to the bundle_list
#
local_make_identity(company_ident_path, company_results_path)

{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--8b1a3f27-0036-4d72-aad5-e5601f5ef165",
    "created": "2023-12-13T14:20:17.494865Z",
    "modified": "2023-12-13T14:20:17.494865Z",
    "name": "Example Company",
    "description": "An Example Organisation",
    "identity_class": "organization",
    "sectors": [
        "technology"
    ]
}


### C.2 Setup Employment Relationship

Since there is no employed-by SRO setup yet, we will use the attributed-to SRO as a stand-in until we implement it

In [9]:
def employ_identities(employer=None, employee=None):
    if employee and employer:
        # Set the Relative Input and Output Paths for the block
        sro_data_rel_path = path_base + sro_data
        sro_results_rel_path = results_base + "employ_SRO_" + employee["name"] + ".json"
        #
        # NOTE: This code is only To fake input ports
        # Add the source and target identities and the reltaionship type
        ##
        if os.path.exists(sro_data_rel_path):
            with open(sro_data_rel_path, "r") as sro_form:
                results_data = json.load(sro_form)
                results_data["source"] = employee
                results_data["target"] = employer
                results_data["relationship_type"] = "attributed-to"
            with open(sro_data_rel_path, 'w') as f:
                f.write(json.dumps(results_data))
        # Make the Identity object
        make_sro(sro_data_rel_path,sro_results_rel_path)
        #
        # Remove Port Emulation if used - Fix the data file so it only has form data
        #
        unwind_ports(sro_data_rel_path)
        # Retrieve the saved file
        if os.path.exists(sro_results_rel_path):
            with open(sro_results_rel_path, "r") as script_input:
                export_data = json.load(script_input)
                export_data_list = export_data["relationship"]
                stix_object = export_data_list[0]
                # convert it into a Stix Object and append to the bundle
                sro = Relationship(**stix_object)
                print(sro.serialize(pretty=True))
                bundle_list.append(sro)

# Find TR_User and Company from bundle_list of Stix objects
employee = {}
employer = {}
for bun in bundle_list:
    if bun.type == "identity" and bun.name == "Trusty Jones":
        employee = json.loads(bun.serialize())
    if bun.type == "identity" and bun.identity_class == "organization":
        employer = json.loads(bun.serialize())

# make the email object
employ_identities(employer, employee)

{
    "type": "relationship",
    "spec_version": "2.1",
    "id": "relationship--5d76b374-a414-4981-8354-b7e82e6332d5",
    "created": "2023-12-13T14:20:17.547947Z",
    "modified": "2023-12-13T14:20:17.547947Z",
    "relationship_type": "attributed-to",
    "source_ref": "identity--5ca6788a-f46a-4647-b563-9b951d4dd1a0",
    "target_ref": "identity--8b1a3f27-0036-4d72-aad5-e5601f5ef165"
}


## D. Create and Employ the 3 Sales Uses

only perform once on initialisation, otherwise you must update the identity object

### D.1 First Create the Complex Identities

In [10]:
for ident in identities_data:
    results_paths = {}
    if ident["who"][:3] == "usr":
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]
        local_make_acct(acct_path, results_path)
        local_make_email(email_path, results_path, results_paths['acct'])
        local_make_identity(ident_path, results_path, email_results=results_paths['email'], acct_results=results_paths['acct'])

{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b",
    "user_id": "79563902",
    "account_login": "nsmith",
    "account_type": "sales,",
    "display_name": "Naive Smith"
}
{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--7165e2a9-671f-585d-b1e1-ca59c671d934",
    "value": "john@example.com",
    "display_name": "John Doe",
    "belongs_to_ref": "user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b"
}
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--d9e4051c-2476-42df-9a80-8da48d5a50cf",
    "created": "2023-12-13T14:20:17.607128Z",
    "modified": "2023-12-13T14:20:17.607128Z",
    "name": "Naive",
    "description": "A Naive Individual",
    "roles": [
        "user",
        "sales"
    ],
    "identity_class": "individual",
    "sectors": [
        "technology"
    ],
    "extensions": {
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
 

### D.2 Now Add Employment SRO's to those 3 Users

In [11]:
# Find TR_User and Company from bundle_list of Stix objects
for bun in bundle_list:
    if bun.type == "identity" and bun.identity_class == "individual":
            if "sales" in bun.roles:
                employee = json.loads(bun.serialize())
                # make the email object
                employ_identities(employer, employee)



{
    "type": "relationship",
    "spec_version": "2.1",
    "id": "relationship--c810e291-137a-4c60-92e4-fe25f7f6d24a",
    "created": "2023-12-13T14:20:17.791705Z",
    "modified": "2023-12-13T14:20:17.791705Z",
    "relationship_type": "attributed-to",
    "source_ref": "identity--d9e4051c-2476-42df-9a80-8da48d5a50cf",
    "target_ref": "identity--8b1a3f27-0036-4d72-aad5-e5601f5ef165"
}
{
    "type": "relationship",
    "spec_version": "2.1",
    "id": "relationship--b8d0e8ae-e20a-431a-b1ab-84f961454481",
    "created": "2023-12-13T14:20:17.821991Z",
    "modified": "2023-12-13T14:20:17.821991Z",
    "relationship_type": "attributed-to",
    "source_ref": "identity--465a01a6-7448-448a-b603-cf9dc2c61884",
    "target_ref": "identity--8b1a3f27-0036-4d72-aad5-e5601f5ef165"
}
{
    "type": "relationship",
    "spec_version": "2.1",
    "id": "relationship--3a2dc6e5-1614-4aa5-9658-2f545d4e473a",
    "created": "2023-12-13T14:20:17.851989Z",
    "modified": "2023-12-13T14:20:17.851989Z",


## E. Create and Save the Bundle of Company Identities


In [12]:
bundle = Bundle(*bundle_list)
bundle_export = json.loads(bundle.serialize())
print(bundle.serialize(pretty=True))
identities_context_mem = "./Context_Mem/identity_bundle.json"
with open(identities_context_mem, 'w') as f:
    f.write(json.dumps(bundle_export))

{
    "type": "bundle",
    "id": "bundle--c0380771-c5e7-42ff-ac68-c357ca7e96db",
    "objects": [
        {
            "type": "user-account",
            "spec_version": "2.1",
            "id": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3",
            "user_id": "79563902",
            "account_login": "tjones",
            "account_type": "soc,",
            "display_name": "Trusty Jones"
        },
        {
            "type": "email-addr",
            "spec_version": "2.1",
            "id": "email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3",
            "value": "trusty@example.com",
            "belongs_to_ref": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3",
            "display_name": "Trusty Jones"
        },
        {
            "type": "identity",
            "spec_version": "2.1",
            "id": "identity--5ca6788a-f46a-4647-b563-9b951d4dd1a0",
            "created": "2023-12-13T14:20:17.443978Z",
            "modified": "2023-12-13T14:20:17.443978Z",
